In [1]:
import glob
import os
import random
from datetime import datetime
from pathlib import Path

import laspy
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

In [2]:
def read_las(pointcloudfile, get_attributes=False, useevery=1):
    """
    :param pointcloudfile: specification of input file (format: las or laz)
    :param get_attributes: if True, will return all attributes in file, otherwise will only return XYZ (default is False)
    :param useevery: value specifies every n-th point to use from input, i.e. simple subsampling (default is 1, i.e. returning every point)
    :return: 3D array of points (x,y,z) of length number of points in input file (or subsampled by 'useevery')
    """

    # Read file
    inFile = laspy.read(pointcloudfile)

    # Get coordinates (XYZ)
    coords = np.vstack((inFile.x, inFile.y, inFile.z)).transpose()
    coords = coords[::useevery, :]

    # Return coordinates only
    if get_attributes == False:
        return coords

    # Return coordinates and attributes
    else:
        las_fields = [info.name for info in inFile.points.point_format.dimensions]
        attributes = {}
        # for las_field in las_fields[3:]:  # skip the X,Y,Z fields
        for las_field in las_fields:  # get all fields
            attributes[las_field] = inFile.points[las_field][::useevery]
        return (coords, attributes)

In [3]:
def rotate_points(coords):
    rotation = np.random.uniform(-180, 180)
    # Convert rotation values to radians
    rotation = np.radians(rotation)

    # Rotate point cloud
    rot_mat = np.array(
        [
            [np.cos(rotation), -np.sin(rotation), 0],
            [np.sin(rotation), np.cos(rotation), 0],
            [0, 0, 1],
        ]
    )

    aug_coords = coords
    aug_coords[:, :3] = np.matmul(aug_coords[:, :3], rot_mat)
    return aug_coords

In [4]:
def point_removal(coords, x=None):
    # Get list of ids
    idx = list(range(np.shape(coords)[0]))
    random.shuffle(idx)  # shuffle ids
    idx = np.random.choice(
        idx, random.randint(round(len(idx) * 0.9), len(idx)), replace=False
    )  # pick points randomly removing 0 - 50 points

    # Remove random values
    aug_coords = coords[idx, :]  # remove coords
    if x is None:  # remove x
        aug_x = aug_coords
    else:
        aug_x = x[idx, :]

    return aug_coords, aug_x

In [5]:
def random_noise(coords, dim, x=None):
    # Random standard deviation value
    random_noise_sd = np.random.uniform(0.01, 0.025)
    print(np.shape(x))

    # Add/Subtract noise
    if np.random.uniform(0, 1) >= 0.5:  # 50% chance to add
        aug_coords = coords + np.random.normal(
            0, random_noise_sd, size=(np.shape(coords)[0], 3)
        )
        if x is None:
            aug_x = aug_coords
        else:
            aug_x = x + np.random.normal(0, random_noise_sd, size=(np.shape(x)))
    else:  # 50% chance to subtract
        aug_coords = coords - np.random.normal(
            0, random_noise_sd, size=(np.shape(coords)[0], 3)
        )
        if x is None:
            aug_x = aug_coords
        else:
            aug_x = x - np.random.normal(0, random_noise_sd, size=(np.shape(x)))

    # Randomly choose between 0 and 50 augmented noise points
    use_idx = np.random.choice(
        aug_coords.shape[0], random.randint(0, round(len(aug_coords) * 0.1)), replace=False
    )
    aug_coords = aug_coords[use_idx, :]  # get random points
    aug_coords = np.append(coords, aug_coords, axis=0)  # add points
    aug_x = aug_x[use_idx, :]  # get random point values
    aug_x = np.append(x, aug_x)  # add random point values

    if dim == 1:
        aug_x = aug_x[:, np.newaxis]

    return aug_coords, aug_x

In [6]:
from matplotlib import pyplot as plt


def plot_3d(coords):
    # Plot parameters
    plt.rcParams["figure.figsize"] = [7.00, 7.00]  # figure size
    plt.rcParams["figure.autolayout"] = True  # auto layout

    # Create figure
    fig = plt.figure()  # initialize figure
    ax = fig.add_subplot(111, projection="3d")  # 3d projection
    x = coords[:, 0]  # x coordinates
    y = coords[:, 1]  # y coordinates
    z = coords[:, 2]  # z coordinates
    ax.scatter(x, y, z, c=z, alpha=1)  # create a scatter plot
    plt.show()  # show plot

In [7]:
class PointCloudsInFiles(InMemoryDataset):
    """Point cloud dataset where one data point is a file."""

    def __init__(
        self, root_dir, glob="*", column_name="", max_points=200_000, use_columns=None
    ):
        """
        Args:
            root_dir (string): Directory with the datasets
            glob (string): Glob string passed to pathlib.Path.glob
            column_name (string): Column name to use as target variable (e.g. "Classification")
            use_columns (list[string]): Column names to add as additional input
        """
        self.files = list(Path(root_dir).glob(glob))
        self.column_name = column_name
        self.max_points = max_points
        if use_columns is None:
            use_columns = []
        self.use_columns = use_columns
        super().__init__()

    def __len__(self):
        # Return length
        return len(self.files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file name
        filename = str(self.files[idx])

        # Read las/laz file
        coords, attrs = read_las(filename, get_attributes=True)

        # Resample number of points to max_points
        if coords.shape[0] >= self.max_points:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=False)
        else:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=True)

        # Get x values
        if len(self.use_columns) > 0:
            x = np.empty((self.max_points, len(self.use_columns)), np.float32)
            for eix, entry in enumerate(self.use_columns):
                x[:, eix] = attrs[entry][use_idx]
        else:
            x = coords[use_idx, :]

        # Get coords
        coords = coords - np.mean(coords, axis=0)  # centralize coordinates

        # impute target
        target = attrs[self.column_name]
        target[np.isnan(target)] = np.nanmean(target)

        # Transform data to tensor
        sample = Data(
            x=torch.from_numpy(x).float(),
            y=torch.from_numpy(
                np.unique(np.array(target[use_idx][:, np.newaxis]))
            ).type(torch.LongTensor),
            pos=torch.from_numpy(coords[use_idx, :]).float(),
        )
        if coords.shape[0] < 100:
            return None
        return sample

In [8]:
class AugmentPointCloudsInFiles(InMemoryDataset):
    """Point cloud dataset where one data point is a file."""

    def __init__(
        self, root_dir, glob="*", column_name="", max_points=200_000, use_columns=None
    ):
        """
        Args:
            root_dir (string): Directory with the datasets
            glob (string): Glob string passed to pathlib.Path.glob
            column_name (string): Column name to use as target variable (e.g. "Classification")
            use_columns (list[string]): Column names to add as additional input
        """
        self.files = list(Path(root_dir).glob(glob))
        self.column_name = column_name
        self.max_points = max_points
        if use_columns is None:
            use_columns = []
        self.use_columns = use_columns
        super().__init__()

    def __len__(self):
        # Return length
        return len(self.files)  # NEED TO ADD MULTIPLICATION FOR NUMBER OF AUGMENTS

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get file name
        filename = str(self.files[idx])

        # Read las/laz file
        coords, attrs = read_las(filename, get_attributes=True)

        # Resample number of points to max_points
        if coords.shape[0] >= self.max_points:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=False)
        else:
            use_idx = np.random.choice(coords.shape[0], self.max_points, replace=True)

        # Get x values
        if len(self.use_columns) > 0:
            x = np.empty((self.max_points, len(self.use_columns)), np.float32)
            for eix, entry in enumerate(self.use_columns):
                x[:, eix] = attrs[entry][use_idx]
        else:
            x = coords[use_idx, :]

        # Get coords
        coords = coords[use_idx, :]
        coords = coords - np.mean(coords, axis=0)  # centralize coordinates

        # Augmentation
        coords, x = point_removal(coords, x)
        coords, x = random_noise(coords, use_columns, x)
        coords = rotate_points(coords)

        # impute target
        target = attrs[self.column_name]
        target[np.isnan(target)] = np.nanmean(target)

        # Transform data to tensor
        sample = Data(
            x=torch.from_numpy(x).float(),
            y=torch.from_numpy(np.unique(np.array(target[:, np.newaxis]))).type(
                torch.LongTensor
            ),
            pos=torch.from_numpy(coords).float(),
        )
        if coords.shape[0] < 100:
            return None
        return sample

In [14]:
train_dataset_path = r"D:\MurrayBrent\data\RMF_ITD\PLOT_LAS\BUF_5M_SC\train"
max_points = 2048
use_columns = ["intensity"]

if train_dataset_path:
    trainset_aug = AugmentPointCloudsInFiles(
        train_dataset_path,
        "*.laz",
        "Class",
        max_points=max_points,
        use_columns=use_columns,
    )

    trainset = PointCloudsInFiles(
        train_dataset_path,
        "*.laz",
        "Class",
        max_points=max_points,
        use_columns=use_columns,
    )

    trainset_con = torch.utils.data.ConcatDataset([trainset, trainset_aug])

In [15]:
from torch_geometric.loader import DataLoader

# train_loader = DataLoader(trainset_con, batch_size=32, shuffle=True, num_workers=0)
train_loader = DataLoader(trainset_con, batch_size=32, shuffle=True, num_workers=0)

In [16]:
for i, data in enumerate(train_loader):
    print(data)

DataBatch(x=[65174, 1], y=[32], pos=[65174, 3], batch=[65174], ptr=[33])
DataBatch(x=[64962, 1], y=[32], pos=[64962, 3], batch=[64962], ptr=[33])
DataBatch(x=[65189, 1], y=[32], pos=[65189, 3], batch=[65189], ptr=[33])
DataBatch(x=[65254, 1], y=[32], pos=[65254, 3], batch=[65254], ptr=[33])
DataBatch(x=[65133, 1], y=[32], pos=[65133, 3], batch=[65133], ptr=[33])
DataBatch(x=[65104, 1], y=[32], pos=[65104, 3], batch=[65104], ptr=[33])
DataBatch(x=[65206, 1], y=[32], pos=[65206, 3], batch=[65206], ptr=[33])
DataBatch(x=[65201, 1], y=[32], pos=[65201, 3], batch=[65201], ptr=[33])
DataBatch(x=[65149, 1], y=[32], pos=[65149, 3], batch=[65149], ptr=[33])
DataBatch(x=[65115, 1], y=[32], pos=[65115, 3], batch=[65115], ptr=[33])
DataBatch(x=[60902, 1], y=[30], pos=[60902, 3], batch=[60902], ptr=[31])


In [ ]:
for i, data in enumerate(train_loader):
    print(data)